In [10]:
CUSTOMERS = {
    1: {"id": 1, "name": "Alice", "email": "alice@x.com"},
    2: {"id": 2, "name": "Bob",   "email": "bob@x.com"},
    3: {"id": 3, "name": "Asha",  "email": "asha@x.com"},
    4: {"id": 4, "name": "Asha1",  "email": "asha1@x.com"},
}
NEXT_ID = 5
[v["name"] for v in CUSTOMERS.values() if "asha" in v["email"].lower()]

['Asha', 'Asha1']

In [11]:
import strawberry

@strawberry.type
class Customer:
    id: int
    name: str
    email: str


In [12]:
# Input types (like DTOs)
@strawberry.input
class CustomerCreateInput:
    name: str
    email: str

@strawberry.input
class CustomerUpdateInput:
    name: str | None = None
    email: str | None = None

In [13]:
@strawberry.type
class Query:
    @strawberry.field
    def customer(self, id: int) -> Customer | None:
        data = CUSTOMERS.get(id)
        return Customer(**data) if data else None

    @strawberry.field
    def customers(self, name_contains: str | None = None) -> list[Customer]:
        rows = [Customer(**row) for row in CUSTOMERS.values()]
        if name_contains:
            needle = name_contains.lower()
            rows = [c for c in rows if needle in c.name.lower()]
        return rows

schema = strawberry.Schema(query=Query)


In [14]:
@strawberry.type
class Mutation:
    @strawberry.mutation
    def create_customer(self, input: CustomerCreateInput) -> Customer:
        global NEXT_ID
        cid = NEXT_ID
        NEXT_ID += 1

        row = {"id": cid, "name": input.name, "email": input.email}
        CUSTOMERS[cid] = row
        return Customer(**row)

    @strawberry.mutation
    def update_customer(self, id: int, input: CustomerUpdateInput) -> Customer | None:
        row = CUSTOMERS.get(id)
        if not row:
            return None

        # patch semantics (only update fields that are provided)
        if input.name is not None:
            row["name"] = input.name
        if input.email is not None:
            row["email"] = input.email

        return Customer(**row)

    @strawberry.mutation
    def delete_customer(self, id: int) -> bool:
        return CUSTOMERS.pop(id, None) is not None

In [15]:
schema = strawberry.Schema(query=Query, mutation=Mutation)

In [16]:
result = schema.execute_sync("""
mutation {
  createCustomer(input: { name: "Asha", email: "asha@x.com" }) {
    id
    name
    email
  }
}
""")
print(result.data)
print(result.errors)


{'createCustomer': {'id': 5, 'name': 'Asha', 'email': 'asha@x.com'}}
None


In [20]:
result = schema.execute_sync("""
mutation {
  updateCustomer(id: 1, input: { email: "alice_new1@x.com" }) {
    id
    name
    email
  }
}
""")
print(result.data)
print(result.errors)

{'updateCustomer': {'id': 1, 'name': 'Alice', 'email': 'alice_new1@x.com'}}
None


In [18]:
result = schema.execute_sync("""
mutation {
  deleteCustomer(id: 2)
}
""")
print(result.data)
print(result.errors)


{'deleteCustomer': True}
None


In [21]:
print(CUSTOMERS)

{1: {'id': 1, 'name': 'Alice', 'email': 'alice_new1@x.com'}, 3: {'id': 3, 'name': 'Asha', 'email': 'asha@x.com'}, 4: {'id': 4, 'name': 'Asha1', 'email': 'asha1@x.com'}, 5: {'id': 5, 'name': 'Asha', 'email': 'asha@x.com'}}
